In [5]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from omnetpp.scave import results, chart, utils

relPath = "../results/"
cliMeasNames = ["measureReceivedAmount", "readSent", "commandSent", "readResponse", 
             "commandResponse", "readResponseTimeout", "commandResponseTimeout"]
evilMeasNames = ["measureBlock", "measureCompromised", "readRequestBlock", "readRequestCompromised", 
                 "commandRequestBlock", "commandRequestCompromised", "readResponseBlock", "readResponseCompromised",
                 "commandResponseBlock", "commandResponseCompromised"]

rep = 10
fileNames = []

name = "MMS_MITM_Wired_Lan_Multi_Server"
for i in range(1,10):
    fileNames.append(relPath + name + '-#' + i.__str__() + '.sca')

print(fileNames)


['../results/MMS_MITM_Wired_Lan_Multi_Server-#1.sca', '../results/MMS_MITM_Wired_Lan_Multi_Server-#2.sca', '../results/MMS_MITM_Wired_Lan_Multi_Server-#3.sca', '../results/MMS_MITM_Wired_Lan_Multi_Server-#4.sca', '../results/MMS_MITM_Wired_Lan_Multi_Server-#5.sca', '../results/MMS_MITM_Wired_Lan_Multi_Server-#6.sca', '../results/MMS_MITM_Wired_Lan_Multi_Server-#7.sca', '../results/MMS_MITM_Wired_Lan_Multi_Server-#8.sca', '../results/MMS_MITM_Wired_Lan_Multi_Server-#9.sca']


In [6]:
data = results.read_result_files(fileNames)
scl = results.get_scalars(data)
